In [1]:
from backend.app.results.automatic_cost_calc import cost_calculation_from_energysystem
import os

from oemof import solph

simulation_token = "32e5b1d8-4b49-414f-b336-20557b2b4113"
simulations_path = os.path.abspath(
    os.path.join(os.getcwd(), '..', 'storage', 'simulations', simulation_token, "dump")
)
print(simulations_path)

dump_path = os.path.join(simulations_path, simulation_token, "dump", "oemof_es.dump")
# Prüfen Sie, ob die Datei existiert und nicht leer ist
if os.path.exists(os.path.join(simulations_path, "oemof_es.dump")):
    file_size = os.path.getsize(os.path.join(simulations_path, "oemof_es.dump"))
    print(f"Dateigröße: {file_size} Bytes")

es = solph.EnergySystem()
es.restore(
    dpath=os.path.join(simulations_path),
    filename='oemof_es.dump'
)

### Ab hier kopieren
components = []
busses = []
result_data = []

for node in es.nodes:
    if isinstance(node, solph.Bus):
        print(f"Bus: {node}")
        busses.append(node)
    else:
        components.append(node)

costs = cost_calculation_from_energysystem(es)
print(f"Costs: {costs}")

#print(result_data)


/Users/pyrokar/Documents/Coding/ensys-gui-new/storage/simulations/32e5b1d8-4b49-414f-b336-20557b2b4113/dump
Dateigröße: 360001 Bytes
Bus: electricity
Costs:       investment costs  variable costs  profits
grid               NaN      1224.75402      NaN


/Users/pyrokar/Documents/Coding/ensys-gui-new/.venv/lib/python3.12/site-packages/oemof/network/energy_system.py:205: ExperimentalFeatureWarning: The API to access nodes by label is experimental and might change without prior notice.
  warnings.warn(msg, debugging.ExperimentalFeatureWarning)
